In [1]:
%load_ext autoreload
%autoreload 2

## Dataset loading

In [27]:
with open('dataset/wsj_0001.dp') as f:
    for line in f:
        print(line, next(f))
        #print(next(iter(f)))
#lines

Pierre	NNP	2
 Vinken	NNP	8

,	,	2
 61	CD	5

years	NNS	6
 old	JJ	2

,	,	2
 will	MD	0

join	VB	8
 the	DT	11

board	NN	9
 as	IN	9

a	DT	15
 nonexecutive	JJ	15

director	NN	12
 Nov.	NNP	9

29	CD	16
 .	.	8


 Mr.	NNP	2

Vinken	NNP	3
 is	VBZ	0

chairman	NN	3
 of	IN	4

Elsevier	NNP	7
 N.V.	NNP	12

,	,	12
 the	DT	12

Dutch	NNP	12
 publishing	VBG	12

group	NN	5
 .	.	3



In [52]:
from utils.dataset_loader import load_datasets

((dataset_documents_train, dataset_labels_train), (dataset_documents_val, dataset_labels_val), 
            (dataset_documents_test, dataset_labels_test)) = load_datasets(folder_path='dataset', divide_by_sentence=True)

In [53]:
dataset_documents_train[2]

'rudolph agnew 55 years old and former chairman of consolidated gold fields plc was named a nonexecutive director of this british industrial conglomerate'

In [54]:
dataset_labels_train[2]

'NNP NNP CD NNS JJ CC JJ NN IN NNP NNP NNP NNP VBD VBN DT JJ NN IN DT JJ JJ NN'

In [65]:
from tensorflow.keras.layers import TextVectorization

In [66]:
vectorizer = TextVectorization()

In [67]:
vectorizer.adapt(data=dataset_documents_train)

In [69]:
vectorizer(dataset_documents_train)

<tf.Tensor: shape=(1960, 171), dtype=int64, numpy=
array([[4690, 2334, 2273, ...,    0,    0,    0],
       [  15, 2334,   11, ...,    0,    0,    0],
       [1845, 6954,  944, ...,    0,    0,    0],
       ...,
       [   6,   39,  885, ...,    0,    0,    0],
       [  34, 5314, 5079, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0]], dtype=int64)>

In [70]:
vectorizer.get_vocabulary()

['',
 '[UNK]',
 'the',
 'of',
 'a',
 'to',
 'in',
 'and',
 'for',
 's',
 'that',
 'is',
 'said',
 'it',
 'on',
 'mr',
 'at',
 'with',
 'by',
 'as',
 'was',
 'has',
 'are',
 'nt',
 'have',
 'its',
 'he',
 'us',
 'new',
 'but',
 'an',
 'from',
 'be',
 'they',
 'says',
 'will',
 'about',
 'million',
 'or',
 'this',
 'company',
 'their',
 'who',
 'more',
 'year',
 'were',
 'which',
 'japanese',
 'than',
 'had',
 'would',
 'she',
 'one',
 'also',
 'not',
 'been',
 'president',
 'years',
 'market',
 'billion',
 'up',
 'some',
 'japan',
 'corp',
 'first',
 'other',
 'i',
 'his',
 'because',
 'two',
 'we',
 'trading',
 'there',
 'mrs',
 'can',
 'all',
 'stock',
 'last',
 'no',
 'many',
 'if',
 'york',
 'so',
 'say',
 'inc',
 'when',
 'such',
 'could',
 'any',
 'these',
 'her',
 'co',
 'time',
 'into',
 'funds',
 'do',
 'after',
 'rrb',
 'out',
 'even',
 'only',
 'companies',
 'bank',
 'over',
 'most',
 'lrb',
 'test',
 'research',
 'next',
 'business',
 'what',
 'share',
 'prices',
 'now',
 'g

In [59]:
tokenizer.fit_on_texts(dataset_documents_train)

In [63]:
tokenizer.texts_to_sequences(dataset_documents_train)[2]

[1784,
 3410,
 748,
 61,
 163,
 7,
 528,
 172,
 3,
 2325,
 1171,
 593,
 1785,
 20,
 472,
 4,
 1170,
 290,
 3,
 42,
 1172,
 473,
 3411]

In [51]:
tokenizer[dataset_documents_train[0]]

TypeError: 'Tokenizer' object is not subscriptable